In [2]:
import pandas as pd

df = pd.read_parquet('../data/03_primary/preprocessed_customers.parquet')
print(df.head())

   gender  SeniorCitizen  Partner  Dependents    tenure  PhoneService  \
0       0              0        1           0 -1.277445             0   
1       1              0        0           0  0.066327             1   
2       1              0        0           0 -1.236724             1   
3       1              0        0           0  0.514251             0   
4       0              0        0           0 -1.236724             1   

   PaperlessBilling  MonthlyCharges  TotalCharges  Churn  ...  \
0                 1       -1.160323     -0.992611      0  ...   
1                 0       -0.259629     -0.172165      0  ...   
2                 1       -0.362660     -0.958066      1  ...   
3                 0       -0.746535     -0.193672      0  ...   
4                 1        0.197365     -0.938874      1  ...   

   TechSupport_Yes  StreamingTV_No internet service  StreamingTV_Yes  \
0            False                            False            False   
1            False        

In [ ]:
# ============================================
# ANALYZE PREPROCESSED DATA STRUCTURE
# ============================================
print("=" * 50)
print("PREPROCESSED DATA OVERVIEW")
print("=" * 50)
print(f"Shape: {df.shape}")
print(f"\nColumns ({len(df.columns)}):")
print(df.columns.tolist())
print(f"\nData types:")
print(df.dtypes.value_counts())

In [ ]:
# Identify column groups for feature engineering
binary_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
target_col = 'Churn'

# One-hot encoded service columns (identify by patterns)
service_cols = [col for col in df.columns if any(x in col for x in 
    ['MultipleLines_', 'InternetService_', 'OnlineSecurity_', 'OnlineBackup_',
     'DeviceProtection_', 'TechSupport_', 'StreamingTV_', 'StreamingMovies_'])]

contract_cols = [col for col in df.columns if 'Contract_' in col]
payment_cols = [col for col in df.columns if 'PaymentMethod_' in col]

print("Service columns:", service_cols)
print("\nContract columns:", contract_cols)
print("\nPayment columns:", payment_cols)

In [ ]:
# ============================================
# FEATURE 1: SERVICE COUNT FEATURES
# ============================================
print("=" * 50)
print("CREATING SERVICE COUNT FEATURES")
print("=" * 50)

# Count of "Yes" services (excluding "No internet service" options)
yes_service_cols = [col for col in df.columns if col.endswith('_Yes') and 
                    any(x in col for x in ['OnlineSecurity', 'OnlineBackup', 
                    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'])]

print(f"Yes service columns: {yes_service_cols}")

# Total services count
df['total_services'] = df[yes_service_cols].sum(axis=1)

# Add PhoneService and check for internet (not "No internet service")
df['total_services'] = df['total_services'] + df['PhoneService']

# Has internet service (if any internet-related Yes service exists or no "No internet service")
no_internet_cols = [col for col in df.columns if 'No internet service' in col]
df['has_internet'] = (~df[no_internet_cols].any(axis=1)).astype(int)
df['total_services'] = df['total_services'] + df['has_internet']

print(f"\nTotal services distribution:")
print(df['total_services'].value_counts().sort_index())

In [ ]:
# Additional service-related features

# Has streaming services (TV or Movies)
streaming_cols = [col for col in df.columns if ('StreamingTV_Yes' in col or 'StreamingMovies_Yes' in col)]
df['has_streaming'] = df[streaming_cols].any(axis=1).astype(int)

# Has security/protection services
security_cols = [col for col in df.columns if any(x in col for x in 
                ['OnlineSecurity_Yes', 'OnlineBackup_Yes', 'DeviceProtection_Yes', 'TechSupport_Yes'])]
df['has_security_services'] = df[security_cols].any(axis=1).astype(int)
df['security_services_count'] = df[security_cols].sum(axis=1)

# Has multiple lines
multiple_lines_cols = [col for col in df.columns if 'MultipleLines_Yes' in col]
if multiple_lines_cols:
    df['has_multiple_lines'] = df[multiple_lines_cols].any(axis=1).astype(int)
else:
    df['has_multiple_lines'] = 0

print("Service features created:")
print(f"- total_services: {df['total_services'].nunique()} unique values")
print(f"- has_internet: {df['has_internet'].value_counts().to_dict()}")
print(f"- has_streaming: {df['has_streaming'].value_counts().to_dict()}")
print(f"- has_security_services: {df['has_security_services'].value_counts().to_dict()}")
print(f"- security_services_count: {df['security_services_count'].value_counts().sort_index().to_dict()}")

In [ ]:
# ============================================
# FEATURE 2: TENURE-BASED FEATURES
# ============================================
print("=" * 50)
print("CREATING TENURE-BASED FEATURES")
print("=" * 50)

# Note: tenure is already standardized, so we use relative thresholds
# Tenure quartiles for grouping
tenure_q25 = df['tenure'].quantile(0.25)
tenure_q50 = df['tenure'].quantile(0.50)
tenure_q75 = df['tenure'].quantile(0.75)

print(f"Tenure quartiles (scaled): Q25={tenure_q25:.3f}, Q50={tenure_q50:.3f}, Q75={tenure_q75:.3f}")

# Tenure group (categorical based on quartiles)
def get_tenure_group(tenure):
    if tenure <= tenure_q25:
        return 0  # New customer
    elif tenure <= tenure_q50:
        return 1  # Short-term
    elif tenure <= tenure_q75:
        return 2  # Mid-term
    else:
        return 3  # Long-term

df['tenure_group'] = df['tenure'].apply(get_tenure_group)

# Is new customer (bottom 25% of tenure)
df['is_new_customer'] = (df['tenure'] <= tenure_q25).astype(int)

# Is loyal customer (top 25% of tenure)
df['is_loyal_customer'] = (df['tenure'] > tenure_q75).astype(int)

print(f"\nTenure group distribution:")
print(df['tenure_group'].value_counts().sort_index())
print(f"\nNew customers: {df['is_new_customer'].sum()} ({df['is_new_customer'].mean()*100:.1f}%)")
print(f"Loyal customers: {df['is_loyal_customer'].sum()} ({df['is_loyal_customer'].mean()*100:.1f}%)")

In [ ]:
# ============================================
# FEATURE 3: CONTRACT & PAYMENT RISK FEATURES
# ============================================
print("=" * 50)
print("CREATING CONTRACT & PAYMENT RISK FEATURES")
print("=" * 50)

# Month-to-month contract is high risk (no long-term commitment)
# If neither One year nor Two year, then it's month-to-month
df['is_month_to_month'] = (~(df['Contract_One year'] | df['Contract_Two year'])).astype(int)

# Has long-term contract (1 or 2 years)
df['has_long_contract'] = (df['Contract_One year'] | df['Contract_Two year']).astype(int)

# Electronic check payment is often associated with higher churn
electronic_check_col = [col for col in df.columns if 'Electronic check' in col]
if electronic_check_col:
    df['uses_electronic_check'] = df[electronic_check_col[0]].astype(int)
else:
    df['uses_electronic_check'] = 0

# Automatic payment (lower churn risk)
auto_payment_cols = [col for col in df.columns if 'automatic' in col.lower()]
if auto_payment_cols:
    df['has_auto_payment'] = df[auto_payment_cols].any(axis=1).astype(int)
else:
    df['has_auto_payment'] = 0

print("Contract & Payment features created:")
print(f"- is_month_to_month: {df['is_month_to_month'].value_counts().to_dict()}")
print(f"- has_long_contract: {df['has_long_contract'].value_counts().to_dict()}")
print(f"- uses_electronic_check: {df['uses_electronic_check'].value_counts().to_dict()}")
print(f"- has_auto_payment: {df['has_auto_payment'].value_counts().to_dict()}")

In [ ]:
# ============================================
# FEATURE 4: CHARGE-RELATED FEATURES
# ============================================
print("=" * 50)
print("CREATING CHARGE-RELATED FEATURES")
print("=" * 50)

# Charge per service (normalized charges / total services)
# Avoid division by zero
df['charge_per_service'] = df['MonthlyCharges'] / (df['total_services'] + 1)

# High monthly charges flag (above 75th percentile)
monthly_q75 = df['MonthlyCharges'].quantile(0.75)
df['is_high_charges'] = (df['MonthlyCharges'] > monthly_q75).astype(int)

# Charge-tenure ratio (how much they pay relative to tenure)
# Higher ratio might indicate newer customers paying more
df['charge_tenure_ratio'] = df['MonthlyCharges'] / (df['tenure'] + 1e-6)

print("Charge features created:")
print(f"- charge_per_service: mean={df['charge_per_service'].mean():.3f}, std={df['charge_per_service'].std():.3f}")
print(f"- is_high_charges: {df['is_high_charges'].value_counts().to_dict()}")
print(f"- charge_tenure_ratio: mean={df['charge_tenure_ratio'].mean():.3f}, std={df['charge_tenure_ratio'].std():.3f}")

In [ ]:
# ============================================
# FEATURE 5: INTERACTION & RISK FEATURES
# ============================================
print("=" * 50)
print("CREATING INTERACTION & RISK FEATURES")
print("=" * 50)

# High risk combination: new customer + month-to-month + no security
df['high_risk_combo'] = (
    (df['is_new_customer'] == 1) & 
    (df['is_month_to_month'] == 1) & 
    (df['has_security_services'] == 0)
).astype(int)

# Churn risk score (sum of risk factors)
df['churn_risk_score'] = (
    df['is_new_customer'] + 
    df['is_month_to_month'] + 
    df['uses_electronic_check'] + 
    df['is_high_charges'] +
    (1 - df['has_security_services']) +  # No security = risk
    df['PaperlessBilling']  # Paperless billing often correlates with churn
)

# Low engagement: few services + no long contract
df['low_engagement'] = (
    (df['total_services'] <= 2) & 
    (df['has_long_contract'] == 0)
).astype(int)

# Senior citizen with high charges (potentially price-sensitive)
df['senior_high_charges'] = (
    (df['SeniorCitizen'] == 1) & 
    (df['is_high_charges'] == 1)
).astype(int)

# Family indicator (has partner OR dependents)
df['has_family'] = ((df['Partner'] == 1) | (df['Dependents'] == 1)).astype(int)

print("Interaction features created:")
print(f"- high_risk_combo: {df['high_risk_combo'].sum()} customers ({df['high_risk_combo'].mean()*100:.1f}%)")
print(f"- churn_risk_score distribution:\n{df['churn_risk_score'].value_counts().sort_index()}")
print(f"- low_engagement: {df['low_engagement'].sum()} customers ({df['low_engagement'].mean()*100:.1f}%)")
print(f"- senior_high_charges: {df['senior_high_charges'].sum()} customers")
print(f"- has_family: {df['has_family'].sum()} customers ({df['has_family'].mean()*100:.1f}%)")

In [ ]:
# ============================================
# FEATURE ENGINEERING SUMMARY
# ============================================
print("=" * 60)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 60)

# List of new engineered features
new_features = [
    # Service features
    'total_services', 'has_internet', 'has_streaming', 
    'has_security_services', 'security_services_count', 'has_multiple_lines',
    # Tenure features
    'tenure_group', 'is_new_customer', 'is_loyal_customer',
    # Contract & Payment features
    'is_month_to_month', 'has_long_contract', 'uses_electronic_check', 'has_auto_payment',
    # Charge features
    'charge_per_service', 'is_high_charges', 'charge_tenure_ratio',
    # Interaction features
    'high_risk_combo', 'churn_risk_score', 'low_engagement', 
    'senior_high_charges', 'has_family'
]

print(f"\nTotal new features created: {len(new_features)}")
print(f"Original features: {31}")
print(f"Final feature count: {len(df.columns)}")

print("\n--- NEW FEATURES LIST ---")
for i, feat in enumerate(new_features, 1):
    if feat in df.columns:
        print(f"{i:2d}. {feat}")

print(f"\n--- FINAL DATAFRAME SHAPE ---")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

In [ ]:
# ============================================
# FEATURE VALIDATION: CORRELATION WITH TARGET
# ============================================
print("=" * 60)
print("FEATURE CORRELATION WITH CHURN (Target)")
print("=" * 60)

# Calculate correlation of new features with Churn
correlations = df[new_features + ['Churn']].corr()['Churn'].drop('Churn').sort_values(ascending=False)

print("\nFeatures positively correlated with Churn (higher = more likely to churn):")
positive_corr = correlations[correlations > 0]
for feat, corr in positive_corr.items():
    print(f"  {feat}: {corr:.4f}")

print("\nFeatures negatively correlated with Churn (lower = less likely to churn):")
negative_corr = correlations[correlations < 0]
for feat, corr in negative_corr.items():
    print(f"  {feat}: {corr:.4f}")

In [ ]:
# ============================================
# SAVE ENGINEERED FEATURES
# ============================================
print("=" * 60)
print("SAVING ENGINEERED FEATURES")
print("=" * 60)

# Save to parquet
output_path = '../data/04_feature/features_engineered.parquet'
df.to_parquet(output_path, index=False)
print(f"Saved to: {output_path}")
print(f"Shape: {df.shape}")

# Preview final dataframe
print("\nFinal DataFrame preview:")
df.head()